In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import *

In [ ]:

Numsize=500
feature=165
lrate=0.1
batch_size = 100#训练batch的大小
steps=5000
dimension=165
total=400

In [ ]:
inputX= pd.read_csv('xDataR1.txt',header=None,sep=',')
inputY=pd.read_csv('yDataR1.txt',header=None,sep=',')

x_data =inputX.values#[:,0:dimension]
y_data=inputY.values#[:,0:dimension]

train_start = 0
train_end = int(np.floor(0.8*Numsize))
test_start = train_end 
test_end = Numsize

xdata_train = x_data[np.arange(train_start, train_end), :]
xdata_test = x_data[np.arange(test_start, test_end), :]
ydata_train =y_data[np.arange(train_start, train_end), :]
ydata_test = y_data[np.arange(test_start, test_end), :]

# xdata_train = np.transpose(x_data[np.arange(train_start, train_end), :])
# xdata_test = np.transpose(x_data[np.arange(test_start, test_end), :])
# ydata_train =np.transpose(y_data[np.arange(train_start, train_end), :])
# ydata_test = np.transpose(y_data[np.arange(test_start, test_end), :])

# print(xdata_train)
# print(ydata_train)
#xmean=np.mean(x_data[:,0])
# print(xmean)
#xt=np.transpose(x_data)
#yt=np.transpose(y_data)
# print(xt.dtype)
# print(xt.shape)
# print(y_data[:,0:3].shape)
# noise = np.random.normal(0, 0.05, x_data.shape) 

In [ ]:
 # 2.定义节点准备接收数据 # define placeholder for inputs to network 
# xs = tf.placeholder(tf.float32, [dimension,None]) 
# ys = tf.placeholder(tf.float32, [dimension,None]) 
# global_step=tf.Variable(0)
# learning_rate=tf.train.exponential_decay(0.1,global_step,100,0.96,staircase=True)
# learning_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
xs = tf.placeholder(tf.float32, [None,dimension],name="x_traindata") 
ys = tf.placeholder(tf.float32, [None,dimension],name="y_traindata") 

In [ ]:
# def add_layer(inputs, in_size, out_size, activation_function=None): 
#     # add one more layer and return the output of this layer 
#     Weights = tf.Variable(tf.random_normal([in_size, out_size])) 
#     biases = tf.Variable(tf.zeros([1, out_size]) + 0.1) 
#     Wx_plus_b = tf.matmul(inputs, Weights) + biases 
#     if activation_function is None: 
#         outputs = Wx_plus_b
#     else:
#         outputs = activation_function(Wx_plus_b) 
#     return outputs
def add_layer(n_layer,inputs, in_size, out_size,Wname,Bname, activation_function):
    # 构建权重 : in_size * out)_sieze 大小的矩阵
    layer_name = 'layer%s'% n_layer
    with tf.name_scope('layer_name'):
        with tf.name_scope('Weights'):
            weights = tf.Variable(tf.random_normal([ in_size,out_size]),name=Wname)
            # 构建偏置 : 1 * out_size 的矩阵
            biases = tf.Variable((tf.zeros([1,out_size]) + 0.1),name=Bname)
            # 矩阵相乘
            Wx_plus_b = tf.matmul(inputs, weights) + biases
            if activation_function is None:
                outputs = Wx_plus_b
            else:
                outputs = activation_function(Wx_plus_b)

    return weights,outputs  # 得到输出数据

In [ ]:
#19~30
hidden_layers = 256
# 构建输入层到隐藏层,假设隐藏层有 hidden_layers 个神经元
w1,h1 = add_layer("1",xs, dimension, hidden_layers,"w1","b1",activation_function=None)#
# 构建隐藏层到隐藏层
w2,h2 = add_layer("2",h1, hidden_layers, hidden_layers,"w2","b2", activation_function=tf.nn.relu)
w3,h3 = add_layer("3",h2, hidden_layers, hidden_layers, "w3","b3",activation_function=tf.nn.relu)
# 构建隐藏层到隐藏层
w4,h4= add_layer("4",h3, hidden_layers, hidden_layers,"w4","b4", activation_function=tf.nn.tanh)
# 构建隐藏层到输出层
w5,prediction = add_layer("5",h2, hidden_layers, dimension,"w5","b5", activation_function=None)


In [ ]:
loss =tf.reduce_mean(tf.abs(ys-prediction))#tf.losses.mean_squared_error(ys, prediction)#)#-tf.reduce_mean(ys*tf.log(tf.clip_by_value(prediction,1e-10,1)))#(tf.abs(tf.subtract(prediction,ys)))#reduce_sum(tf.square(ys - prediction), reduction_indices=[1]))
#loss=-tf.reduce_mean()
train_step =tf.train. AdamOptimizer(lrate).minimize(loss)
#tf.train.GradientDescentOptimizer(lrate).minimize(loss) # 优化算法选取SGD,随机梯度下降

# print('将计算图写入事件文件,在TensorBoard里查看')
# writer = tf.summary.FileWriter(logdir='logs/8_2_BP', graph=tf.get_default_graph())
# writer.close()

In [ ]:
cost_history=[ ]
saver = tf.train.Saver()
# steps=50000
#x_test=x_data
# print(x_test)
#[136.0,-84.26, -38.635, 69.61, -142.55, -14.61, 59.635, -140.035, 25.045, 77.69, -142.27, 43.02, 59.51, -143.44, 61.36, 136.61, -136.045, 20.145, 92.865, -151.43, -22.895, 71.68, -140.845, -1.355, 88.0, -150.035, -16.52, 63.665, -134.15, -47.16, 28.955, -115.135, 27.655, 55.01, -121.925, -53.045, 102.315, -144.815, -34.65, 60.9, -122.43, -32.055, 88.52, -146.62, -32.795, 118.35, -123.745, -13.82, 36.915, -85.725, -16.045, 116.85, -123.895, -32.2, 97.35, -143.98, 42.14, 39.345, -132.91, 43.0, 136.75, -136.49, 42.325, 56.64, -112.515, -29.345, 105.12, -141.42, -1.635, 19.855, -92.22, 19.705, 83.51, -151.065, -23.1, 148.68, -119.16, 27.915, 141.095, -88.705, -14.95, 138.705, -136.3, 51.515, 138.185, -128.19, 2.34, 36.355, -78.32, -16.055, 80.18, -154.83, 19.04, 88.845, -150.06, 66.835, 106.915, -143.455, -14.235, 36.45, -130.525, 50.735, 83.205, -147.305, 48.25, 87.665, -162.82, 24.23, 96.09, -155.33, 19.49, 116.89, -145.815, 60.35, 74.375, -144.24, -34.53, 39.005, -125.775, 2.565, 39.86, -132.7, 19.665, 88.27, -148.525, 49.515, 121.615, -112.78, -27.765, 113.195, -135.835, -45.74, 157.565, -97.315, 20.24, 88.34, -149.025, -6.84, 141.47, -83.24, -13.785, 87.93, -152.63, 35.9, 116.975, -141.695, 23.95, 122.585, -124.015, -52.39, 42.64, -81.37, -41.74, 91.4, -148.22, 48.55, 56.545, -115.245, -12.265, 120.055, -114.925, -11.195, 59.325, -124.12, -15.02 ]
with tf.Session() as sess:
    tf.global_variables_initializer().run()  # 初始化所有变量
 
    for i in range(1,steps+1):
       
        start=(i*batch_size)%total
        end=min(start+batch_size,total)
#         if start==end:
#             start=398
#             end=399
        sess.run(train_step, feed_dict={xs: xdata_train[start:end], ys: ydata_train[start:end]})
#         print(xs)
        if i % 20 == 0:
#             m=(i)%100
#             n=(i+1)%100
#             if m>n:
#                 m=n-1
            cost=sess.run(loss, feed_dict={xs: xdata_test, ys: ydata_test})
            #print('w1',sess.run(w1,feed_dict={xs:x_data}))
#             print('第%d次的损失值为'%i,cost)
#             print(xs)
         #   print('h1:',sess.run(h1,feed_dict={xs:x_data}))
#             print('h2:',sess.run(w2,feed_dict={xs:h1}))
#             print('第%d次的损失值为'%i,cost)
#             print(xs)
#             predict=sess.run(prediction,feed_dict={xs:xdata_train[start:end]})
#             hid1=sess.run(h1,feed_dict={xs:xdata_train})
#             hid2=sess.run(h1,feed_dict={xs:xdata_train})
#             print("hid1:\n",hid1)
#             print(xdata_test)
#             print(ydata_test)
            predict=sess.run(prediction,feed_dict={xs:xdata_test})
            print("predict1:",predict)
#             predict=sess.run(prediction,feed_dict={xs:xdata_test[2:3]})
#             print("predict2:",predict)
#             predict=sess.run(prediction,feed_dict={xs:xdata_test[3:4]})
#             print("predict3:",predict)
           # print('h1\n',hid1)
           # print('h2:\n',hid2)
#            print('prediction \n',predict)
            cost_history.append (cost) 
#             print(cost.shape)
    saver.save(sess,"./model.ckpt")
    writer=tf.summary.FileWriter("./log",tf.get_default_graph())
    writer.close()
print(np.min(cost_history))
# with tf.Session() as sess:
#     saver.restore(sess,"../model.ckpt")
    
#     print(sess.run(prediction,feed_dict={xs:xdata_train})
  
plt.plot ( range ( len ( cost_history ) ) ,cost_history )

plt.axis ( [ 0,500,0,np.max ( cost_history ) ] )

plt.xlabel ( 'training epochs' )

plt.ylabel ( 'cost' )

plt.title ( 'cost history' )

plt.show ( ) 

In [ ]:
w = tf.Variable(tf.random_normal([165,165]))
b = tf.Variable(tf.zeros([1,165])+0.1)
prediction = tf.nn.softmax(tf.add(tf.matmul(xs,w),b))
loss=tf.reduce_mean(tf.square(ys-prediction))
train=tf.train.GradientDescentOptimizer(lrate).minimize(loss)

In [ ]:
# x_data=np.linspace(-1,1,165)[400]#[:,np.newaxis] #300*1,输入只有一个神经元
#     X = tf.placeholder (tf.float32, shape = [None, input_size])
#     Y = tf.placeholder (tf.float32, shape = [None, num_classes])import tensorflow as tf



# def weight_variable(shape):
#     initial = tf.truncated_normal(shape, stddev=0.1)
#     return tf.Variable(initial)
# def bias_variable(shape):
#     initial = tf.zeros(shape)
#     return tf.Variable(initial)

# def add_layer(inputs,in_size,out_size,activation_function=None):
#     Weights=weight_variable([in_size,out_size])
#     biases=bias_variable([1,out_size])
#     Wx_plus_b=tf.matmul(inputs,Weights)+biases
#     if activation_function==None:
#         outputs=Wx_plus_b
#     else:
#         outputs=activation_function(Wx_plus_b)
#     return [outputs,Weights]

 # 3.定义神经层：隐藏层和预测层 
 
h1=18
h2=36
h3=72
l_1 = add_layer(xs,dimension,h1,activation_function = tf.nn.relu)
# l_2=add_layer(l_1,h1,h2,activation_function = tf.nn.relu)
prediction = add_layer(l_1,h1,dimension,activation_function = tf.nn.sigmoid)

# l1 = add_layer(xs,dimension,h1,activation_function=tf.nn.relu) 
# # w2,l2=add_layer(l1,h1,h2,activation_function=tf.nn.relu)
# # w3,l3=add_layer(l2,h2,h3,activation_function=tf.nn.relu)
# prediction=add_layer (l1,h1,dimension,activation_function=tf.nn.relu) 

loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices=[1])) 

train_step =tf.train.GradientDescentOptimizer(lrate).minimize(loss)#tf.train.AdamOptimizer(lrate).minimize(loss) # 
# valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
# important step 对所有变量进行初始化 
init = tf.global_variables_initializer()#tf.initialize_all_variables() 
sess = tf.Session() 
# 上面定义的都没有运算，直到 sess.run 才会开始运算 
sess.run(init) 
# 迭代 1000 次学习，
cost_history= [ ] 
for i in range(steps+1): 

    start = max((i * batch_size) % (Numsize),0)
    end = min(start + batch_size,Numsize)
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data}) 
    if i % 50 == 0:
# to see the step improvement
        print ( "After %d iteration:" %i ) 
        cost=sess.run(loss, feed_dict={xs: x_data, ys: y_data})
        print(cost)
#         print(sess.run(w,feed_dict={xs: x_data, ys: y_data}))
        cost_history.append (cost) 
saver = tf.train.Saver()
# print(cost_history)
plt.plot ( range ( len ( cost_history ) ) ,cost_history )

plt.axis ( [ 0,10,0,np.max ( cost_history ) ] )

plt.xlabel ( 'training epochs' )

plt.ylabel ( 'cost' )

plt.title ( 'cost history' )

plt.show ( ) 
# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#     sess.run(init)
#     save_path = saver.save(sess, "my_net/save_net.ckpt")
#     print("Save to path: ", save_path)
# pred_y = sess.run ( y, feed_dict={x: X_test} )

# mse = tf.reduce_mean ( tf.square ( pred_y - y_test ) )

# print ( "MSE: %.4f" % sess.run ( mse ) ) 


In [ ]:
# import tensorflow as tf 
# from  tensorflow.examples.tutorials.mnist  import  input_data
import numpy as np 
from numpy import *
x_data=np.linspace(1,165)#reshape(400,165)#np.linspace(-1,1,165)
x_data=mat(random.randint(50,200,size=(400,165)))
# mnist = input_data.read_data_sets('data', one_hot = True)

In [ ]:
x_data.shape

In [ ]:
np.ones(Numsize*feature)

In [ ]:
# x_data=mat(np.ones(Numsize*feature),size=(Numsize,feature))
x_data=np.ones((total,feature))
#x1=x1.reshape(Numsize,feature)
print(x_data[400:500])
# x_data=mat(random.randint(50,200,size=(Numsize,feature)))
# noise=np.random.normal(0,0.05,x_data.shape)
y_data=x_data*0.3+0.1#np.square(x_data)-0.5#+noise
print(y_data)
xs=tf.placeholder(tf.float32,[None,feature])
ys=tf.placeholder(tf.float32,[None,feature])
#定义隐含层
# l1=add_layer(xs,165,165,activation_function=tf.nn.relu)
# l2=add_layer(l1,165,165,activation_function=tf.nn.relu)

#定义输出层,假设没有任何激活函数
# [prediction,w]=add_layer(xs,165,165,activation_function=None)
Weights = tf.Variable(tf.random_uniform([1], -1.0, 1),(165,1))
biases = tf.Variable(tf.zeros([1]),(1,165))
prediction = Weights*xs + biases
loss=tf.reduce_mean(tf.reduce_sum(tf.square(ys-prediction)))#,reduction_indices=[1]
optimizer=tf.train.GradientDescentOptimizer(lrate)
train=optimizer.minimize(loss)
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):

#         print(start,end)
        sess.run(train,feed_dict={xs:x_data,ys:y_data})
        if i%100==0:
            print(sess.run(loss,feed_dict={xs:x_data,ys:y_data}))
            print(sess.run(Weights,feed_dict={xs:x_data,ys:y_data}))
            

In [ ]:
plt.figure()
for i in range(399):
    plt.scatter(xdata_train[i,:],ydata_train[i,:],marker='o')
    plt.xlabel('x')
    plt.ylabel('y')
plt.show()